In [5]:
import os
import pandas as pd
import numpy as np
os.listdir()

['ROC_for:CN_AD_xgb_25_no_prune_RFE_correct.png',
 'CN_AD_xgb_25_no_prune_RFE_correct_whole_imp.csv',
 'Features_ranked_for_CN_AD_xgb_25_no_prune_RFE_correct.csv',
 'top_genes_SNP.ipynb',
 '.ipynb_checkpoints']

In [82]:
df = pd.read_csv('CN_AD_xgb_25_no_prune_RFE_correct_whole_imp.csv')
df.head()

,Unnamed: 0,features_fold0,importance_fold0,features_fold1,importance_fold1,features_fold2,importance_fold2,features_fold3,importance_fold3,features_fold4,importance_fold4
0,0,AGE,0.017605,AGE,0.019294,AGE,0.013737,AGE,0.016206,AGE,0.015169
1,1,EDU,0.021709,EDU,0.023218,EDU,0.017361,EDU,0.021598,EDU,0.016440
2,2,1_rs1395713_TT,0.018359,5_rs12522957_TT,0.012723,20_rs6116375_CC,0.054854,3_rs1385464_AA,0.019950,10_rs7918915_TT,0.000000
3,3,20_rs6116375_CC,0.017227,4_rs17049579_CA,0.009017,17_rs9674507_TC,0.007073,18_rs1792738_TT,0.000000,10_rs16920492_TC,0.013213
4,4,2_rs7590552_GA,0.013486,20_rs6116375_CC,0.026362,12_rs2906109_TT,0.015798,6_rs4709165_AG,0.006368,1_rs1891740_CA,0.018189


In [104]:
(0.017605+0.019294+0.013737+0.016206+0.015169)/5

0.0164022

In [97]:
FOLDS = 5
def importance_avg(df,threshold=0.01):
    df = pd.read_csv('CN_AD_xgb_25_no_prune_RFE_correct_whole_imp.csv').fillna('0')
    all_feats = []
    for fold in range(FOLDS):
        all_feats = all_feats + list(df['features_fold'+str(fold)])
    all_feats = list(set(all_feats)) # Overall features in all folds
    all_feats = [f for f in all_feats if f != 'empty']
    df_avg = pd.DataFrame(columns=['features','importance'])
    df_avg['features'] = all_feats
    df_avg['importance'] = np.zeros(len(all_feats,))
    for i in range(len(all_feats)):
        tmp_imp = np.zeros(FOLDS,)
        current_feat = df_avg.iloc[i]['features']
        for fold in range(FOLDS):
            temp_df =df[['features_fold'+str(fold),'importance_fold'+str(fold)]]
            if current_feat in list(temp_df['features_fold'+str(fold)]):
#                 print(temp_df[temp_df['features_fold'+str(fold)] == current_feat]['importance_fold'+str(fold)])
                tmp_imp[fold] = temp_df[temp_df['features_fold'+str(fold)] == current_feat]['importance_fold'+str(fold)].iloc[0]
        df_avg.at[i,'features'] = current_feat
        df_avg.at[i,'importance'] = np.mean(tmp_imp)
    df_avg_sorted = df_avg.sort_values(by=['importance'],ascending=False)
    df_avg_sorted = df_avg_sorted[df_avg_sorted['importance']>=threshold]
    return df_avg_sorted

In [101]:
df_out = importance_avg(df,threshold=0.01)

In [102]:
df_out

,features,importance
60,19_rs2075650_AA,0.051724
156,10_rs11253696_AA,0.026265
58,20_rs6116375_CC,0.025180
116,EDU,0.020065
178,10_rs11253696_GG,0.017222
23,AGE,0.016402
28,18_rs618236_AA,0.014528
133,5_rs168825_GG,0.012946
24,14_rs11156803_TT,0.011812
157,22_rs469120_TC,0.010973


In [103]:
df_out.to_csv('final_SNP_ranked_for_CN_AD.csv')